In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
import pandas as pd
df = pd.read_csv("/home/nagendra/system_files/bin/english/english_clean_new.csv")

In [4]:
# prompt: train test split

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'df' is your pandas DataFrame with 'cleaned' and 'is_hateful' columns

train_df, test_df = train_test_split(df, test_size=0.15, random_state=42)  # Adjust test_size and random_state as needed

print(f"Training data size: {len(train_df)}")
print(f"Testing data size: {len(test_df)}")


Training data size: 5383
Testing data size: 951


In [5]:
train_df['cleaned'] = train_df['cleaned'].astype(str).str.strip()  # Ensure all entries are strings and strip whitespace
print(f"Training data size: {len(train_df)}")

Training data size: 5383


In [6]:
test_df['cleaned'] = test_df['cleaned'].astype(str).str.strip()  # Ensure all entries are strings and strip whitespace
print(f"Training data size: {len(test_df)}")

Training data size: 951


In [7]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.utils.rnn import pad_sequence

class CustomTextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get text data, ensure it's a string
        text = self.data.iloc[idx]["cleaned"]
        if not isinstance(text, str):
            text = ""
            
        # Get the label
        label = self.data.iloc[idx]["is_hateful"]

        # Safely evaluate the 'rational_3' column to get the attention vector
        try:
            attention_vector = eval(self.data.iloc[idx]["rational_1"])
            if not isinstance(attention_vector, list):
                attention_vector = []
        except (SyntaxError, NameError):
            attention_vector = []

        # Tokenize the text
        inputs = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors="pt")

        # Convert attention_vector to a tensor and pad if necessary
        attention_vector = torch.tensor(attention_vector, dtype=torch.float)

        
#          # Safely evaluate the 'rational_3' column to get the attention vector
#         try:
#             attention_vector = eval(self.data.iloc[idx]["rational_2"])
#             if not isinstance(attention_vector, list):
#                 attention_vector = []
#         except (SyntaxError, NameError):
#             attention_vector = []
        
 # Tokenize the text
        inputs = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors="pt")
        tokens = self.tokenizer.tokenize(text)

        # Adjust ground truth attention to match subword tokens
        adjusted_ground_truth_attention = self.adjust_ground_truth_attention(text, tokens, attention_vector)

        # Convert to torch tensor and pad to match max_len
        attention_vector = torch.tensor(adjusted_ground_truth_attention, dtype=torch.float)

        padding_length = self.max_len - attention_vector.size(0)

        if padding_length > 0:
            # Pad the attention vector with zeros to match max_len
            attention_vector = torch.cat([attention_vector, torch.zeros(padding_length)], dim=0)
        else:
            # Truncate the attention vector if it's longer than max_len
            attention_vector = attention_vector[:self.max_len]

        # Tokenized inputs
        input_ids = inputs['input_ids'].squeeze(0)  # Remove batch dimension
        attention_mask = inputs['attention_mask'].squeeze(0)
        token_type_ids = inputs['token_type_ids'].squeeze(0)

        # Convert label to tensor
        label = torch.tensor(label, dtype=torch.long)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
            'label': label,
            'attention_vector': attention_vector  # No unsqueeze(0) here
        }
    
    
    def adjust_ground_truth_attention(self, sentence, tokens, ground_truth_attention):
        """
        Adjusts ground truth attention to match the subword tokens generated by the BERT tokenizer.
        If a word is split into subwords, the attention for the original word is repeated for each subword.
        """
        adjusted_attention = []
        word_idx = 0  # Index for words in sentence.split()

        for token in tokens:
            # If token starts with '##', it's a subword, so repeat the last attention value
            if token.startswith("##"):
                adjusted_attention.append(adjusted_attention[-1])
            else:
                # For new words, append the original ground truth attention value
                if word_idx < len(ground_truth_attention):
                    adjusted_attention.append(ground_truth_attention[word_idx])
                    word_idx += 1
                else:
                    # Handle cases where the number of tokens exceeds the attention values
                    adjusted_attention.append(0)

        return adjusted_attention

In [8]:
class BertWithAttentionSupervision(nn.Module):
    def __init__(self, num_labels=2):
        super(BertWithAttentionSupervision, self).__init__()
        self.bert_classifier = BertForSequenceClassification.from_pretrained("google/muril-base-cased", num_labels=num_labels)
        self.attention_loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, input_ids, attention_mask, token_type_ids, labels, ground_truth_attention=None):
        outputs = self.bert_classifier(input_ids=input_ids,
                                       attention_mask=attention_mask,
                                       token_type_ids=token_type_ids,
                                       labels=labels,
                                       output_attentions=True,
                                       return_dict=True)

        classification_loss = outputs.loss
        logits = outputs.logits

        # Extract attention scores from the final layer
        attention_scores = outputs.attentions[-1][:, :, 0, :]  # CLS token attention across all heads
        avg_attention_scores = attention_scores.mean(dim=1)  # Average attention scores across heads

        # Calculate attention loss only if ground_truth_attention is provided
        attention_loss = 0
        if ground_truth_attention is not None:
            if ground_truth_attention.size(0) != avg_attention_scores.size(0):
                raise ValueError(f"Expected ground_truth_attention to be of size {avg_attention_scores.size(0)}, but got {ground_truth_attention.size(0)}")
            attention_loss = self.attention_loss_fn(avg_attention_scores, ground_truth_attention)

        return logits, classification_loss, attention_loss

In [9]:
# Function to train the model
def train_model(model, train_dataloader, eval_dataloader, optimizer, device, alph=0.7, bet=0.3, epochs=30):
    model.train()
    
    acc1 = 0
    acc = 0

    for epoch in range(epochs):
        total_loss = 0
        for batch in train_dataloader:
            optimizer.zero_grad()

            # Move data to device (GPU or CPU)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['label'].to(device)
            ground_truth_attention = batch['attention_vector'].to(device)

            try:
                # Forward pass
                logits, classification_loss, attention_loss = model(input_ids=input_ids,
                                                                attention_mask=attention_mask,
                                                                token_type_ids=token_type_ids,
                                                                labels=labels,
                                                                ground_truth_attention=ground_truth_attention)

                # Total loss with weights
                total_batch_loss = alph * classification_loss + bet * attention_loss

                # Backpropagation
                total_batch_loss.backward()
                optimizer.step()

                # Accumulate total loss
                total_loss += total_batch_loss.item()

            except RuntimeError as e:
                print(f"Error in batch: {batch}")
                raise e

        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{epochs}, Total Loss: {avg_loss:.4f}")

        # Evaluate the model after each epoch
        acc1 = evaluate_model(model, eval_dataloader, device)
        if acc1 > acc:
                acc = acc1
                print(acc1)
            
            # Save the trained model
                torch.save(model.state_dict(), "/home/nagendra/system_files/bin/english/with_rationales/murill/muril_attention_model_new1.pth")
                print("Model updated")
            
            
# Evaluate model
def evaluate_model(model, dataloader, device):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            logits, _, _ = model(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  token_type_ids=token_type_ids,
                                  labels=None,
                                  ground_truth_attention=None)  # Set to None for evaluation
            
            predictions = torch.argmax(logits, dim=-1)
            all_labels.extend(batch['label'].cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())


    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
#     precision = precision_score(all_labels, all_predictions, average='binary')
#     recall = recall_score(all_labels, all_predictions, average='binary')
#     f1 = f1_score(all_labels, all_predictions, average='binary')

#     macro_precision = precision_score(all_labels, all_predictions, average='macro')
#     macro_recall = recall_score(all_labels, all_predictions, average='macro')
#     macro_f1 = f1_score(all_labels, all_predictions, average='macro')
    return accuracy
#     print(f"Accuracy: {accuracy:.4f}")
#     print(f"Precision: {precision:.4f}")
#     print(f"Recall: {recall:.4f}")
#     print(f"F1 Score: {f1:.4f}")
#     print(f"Macro Precision: {macro_precision:.4f}")
#     print(f"Macro Recall: {macro_recall:.4f}")
#     print(f"Macro F1 Score: {macro_f1:.4f}")


# Main function
def main():
    # Initialize tokenizer
    tokenizer = BertTokenizer.from_pretrained("google/muril-base-cased")
    max_len = 128  # Adjust based on your dataset
#     hindi_tokens = train_df['clean_text'].dropna().str.split().explode().unique().tolist()

#     vocab = tokenizer.get_vocab()
#     new_tokens = set(hindi_tokens) - set(vocab.keys())

#     if new_tokens:
#         tokenizer.add_tokens(list(new_tokens))
    
#     tokenizer.save_pretrained("/home/nagendra/system_files/bin/Hindi/with_rationales/muril/tokenizer")

    # Create dataset and dataloader
    dataset = CustomTextDataset(train_df, tokenizer, max_len=max_len)
    train_dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    
    eval_dataset = CustomTextDataset(test_df, tokenizer, max_len=max_len)
    eval_dataloader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

    # Initialize model and optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BertWithAttentionSupervision(num_labels=2).to(device)

    # Resize embeddings after adding new tokens
    model.bert_classifier.resize_token_embeddings(len(tokenizer))

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

    # Train the model
    train_model(model, train_dataloader, eval_dataloader, optimizer, device, alph=0.5, bet=0.5, epochs=100)


def evaluate_loaded_model(device):
    # Load the tokenizer first
    tokenizer = AutoTokenizer.from_pretrained("/home/nagendra/system_files/bin/english/with_rationales/murill/tokenizer")
    
    # Load the saved model for evaluation
    loaded_model = BertWithAttentionSupervision(num_labels=2).to(device)
    
#     # Ensure you resize the embeddings after loading the tokenizer
#     loaded_model.bert_classifier.resize_token_embeddings(len(tokenizer))
    
    loaded_model.load_state_dict(torch.load("/home/nagendra/system_files/bin/english/with_rationales/murill/muril_attention_model_new1.pth"))
    print("Model loaded for evaluation.")

    # Create a separate evaluation dataset and dataloader
    eval_dataset = CustomTextDataset(test_df, tokenizer, max_len=128)
    eval_dataloader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

    # Evaluate the loaded model
    evaluate_model(loaded_model, eval_dataloader, device)

if __name__ == "__main__":
    main()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     evaluate_loaded_model(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Epoch 1/100, Total Loss: 0.6748
0.7802313354363828
Model updated
Epoch 2/100, Total Loss: 0.5823
0.8349106203995794
Model updated
Epoch 3/100, Total Loss: 0.5254
0.8370136698212408
Model updated
Epoch 4/100, Total Loss: 0.4860
0.8412197686645636
Model updated
Epoch 5/100, Total Loss: 0.4463
Epoch 6/100, Total Loss: 0.4345
Epoch 7/100, Total Loss: 0.4103
Epoch 8/100, Total Loss: 0.3980
Epoch 9/100, Total Loss: 0.3950
Epoch 10/100, Total Loss: 0.3906
Epoch 11/100, Total Loss: 0.3832
Epoch 12/100, Total Loss: 0.3785
Epoch 13/100, Total Loss: 0.3721
Epoch 14/100, Total Loss: 0.3743
Epoch 15/100, Total Loss: 0.3717
Epoch 16/100, Total Loss: 0.3672
Epoch 17/100, Total Loss: 0.3624
Epoch 18/100, Total Loss: 0.3570
Epoch 19/100, Total Loss: 0.3536
Epoch 20/100, Total Loss: 0.3514
Epoch 21/100, Total Loss: 0.3508
Epoch 22/100, Total Loss: 0.3507
Epoch 23/100, Total Loss: 0.3635
Epoch 24/100, Total Loss: 0.3666
Epoch 25/100, Total Loss: 0.3599
Epoch 26/100, Total Loss: 0.3510
Epoch 27/100, Total

In [10]:
# with open('/home/zia/shashi/english/with_rationales/muril/output_large.txt', 'w') as file:
#     file.write(cap.stdout)

In [11]:
# muril_attention_model_new.pth (0.4, 0.6, 30) 2e-5, 64

Model loaded for evaluation.
Evaluation Metrics:
Accuracy: 0.8465
Precision: 0.8789
Recall: 0.8699
F1 Score: 0.8744
Macro Precision: 0.8376
Macro Recall: 0.8396
Macro F1 Score: 0.8385

SyntaxError: invalid syntax (4145168576.py, line 3)

In [ ]:
# muril_attention_model_large.pth (0.7, 0.3, 30)

Model loaded for evaluation.
Evaluation Metrics:
Accuracy: 0.8496
Precision: 0.8681
Recall: 0.8904
F1 Score: 0.8791
Macro Precision: 0.8431
Macro Recall: 0.8376
Macro F1 Score: 0.8401

In [ ]:
# muril_attention_model2.pth (0.7, 0.3, 30)

Model loaded for evaluation.
Evaluation Metrics:
Accuracy: 0.8307
Precision: 0.8259
Recall: 0.9178
F1 Score: 0.8694
Macro Precision: 0.8335
Macro Recall: 0.8050
Macro F1 Score: 0.8144

In [ ]:
# muril_attention_model3.pth (0.75, 0.25, 30)

Model loaded for evaluation.
Evaluation Metrics:
Accuracy: 0.8286
Precision: 0.8636
Recall: 0.8562
F1 Score: 0.8598
Macro Precision: 0.8189
Macro Recall: 0.8205
Macro F1 Score: 0.8196

# Evaluation

In [ ]:
# class BertWithAttentionSupervision(nn.Module):
#     def __init__(self, num_labels=2):
#         super(BertWithAttentionSupervision, self).__init__()
#         self.bert_classifier = BertForSequenceClassification.from_pretrained("google/muril-base-cased", num_labels=num_labels)
#         self.attention_loss_fn = nn.BCEWithLogitsLoss()

#     def forward(self, input_ids, attention_mask, token_type_ids, labels, ground_truth_attention=None):
#         outputs = self.bert_classifier(input_ids=input_ids,
#                                        attention_mask=attention_mask,
#                                        token_type_ids=token_type_ids,
#                                        labels=labels,
#                                        output_attentions=True,
#                                        return_dict=True)

#         classification_loss = outputs.loss
#         logits = outputs.logits

#         # Extract attention scores from the final layer
#         attention_scores = outputs.attentions[-1][:, :, 0, :]  # CLS token attention across all heads
#         avg_attention_scores = attention_scores.mean(dim=1)  # Average attention scores across heads

#         # Calculate attention loss only if ground_truth_attention is provided
#         attention_loss = 0
#         if ground_truth_attention is not None:
#             if ground_truth_attention.size(0) != avg_attention_scores.size(0):
#                 raise ValueError(f"Expected ground_truth_attention to be of size {avg_attention_scores.size(0)}, but got {ground_truth_attention.size(0)}")
#             attention_loss = self.attention_loss_fn(avg_attention_scores, ground_truth_attention)

#         return logits, classification_loss, attention_loss

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluate model
def evaluate_model(model, dataloader, device):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            logits, _, _ = model(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  token_type_ids=token_type_ids,
                                  labels=None,
                                  ground_truth_attention=None)  # Set to None for evaluation

            predictions = torch.argmax(logits, dim=-1)
            all_labels.extend(batch['label'].cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='binary')
    recall = recall_score(all_labels, all_predictions, average='binary')
    f1 = f1_score(all_labels, all_predictions, average='binary')

    macro_precision = precision_score(all_labels, all_predictions, average='macro')
    macro_recall = recall_score(all_labels, all_predictions, average='macro')
    macro_f1 = f1_score(all_labels, all_predictions, average='macro')

    print("Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Macro Precision: {macro_precision:.4f}")
    print(f"Macro Recall: {macro_recall:.4f}")
    print(f"Macro F1 Score: {macro_f1:.4f}")


def evaluate_loaded_model(device):
    # Load the tokenizer first
    tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
    
    # Load the saved model for evaluation
    loaded_model = BertWithAttentionSupervision(num_labels=2).to(device)
    
    # Ensure you resize the embeddings after loading the tokenizer
    loaded_model.bert_classifier.resize_token_embeddings(len(tokenizer))
    
    loaded_model.load_state_dict(torch.load("/home/nagendra/system_files/bin/english/with_rationales/murill/muril_attention_model_new.pth"))
    print("Model loaded for evaluation.")

    # Create a separate evaluation dataset and dataloader
    eval_dataset = CustomTextDataset(test_df, tokenizer, max_len=128)
    eval_dataloader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

    # Evaluate the loaded model
    evaluate_model(loaded_model, eval_dataloader, device)

In [10]:
# %%capture cap
evaluate_loaded_model(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1400526/2443557719.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globa

Model loaded for evaluation.


BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Evaluation Metrics:
Accuracy: 0.8465
Precision: 0.8789
Recall: 0.8699
F1 Score: 0.8744
Macro Precision: 0.8376
Macro Recall: 0.8396
Macro F1 Score: 0.8385


In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# loaded_model = BertWithAttentionSupervision(num_labels=2).to(device)
# loaded_model.load_state_dict(torch.load("/home/zia/shashi/english/with_rationales/muril/muril_attention_model.pth",map_location=device))
# loaded_model.eval()

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
# max_len = 128
# eval_dataset = CustomTextDataset(test_df, tokenizer, max_len=max_len)
# eval_dataloader = DataLoader(eval_dataset, batch_size=8, shuffle=False)

# # Evaluate the loaded model
# evaluate_model(loaded_model, eval_dataloader, device)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
# def get_attention_scores(model, tokenizer, text):
#     # Tokenize input text
#     inputs = tokenizer(text, return_tensors='pt').to(device)
#     # input_ids = inputs['input_ids']

#     # Get the attention weights by running a forward pass
#     with torch.no_grad():
#         # Forward pass through the model with output_attentions=True to get attention scores
#         outputs = model(**inputs, labels=None)
#         print(nn.Softmax(outputs[0]))

#     # Extract attentions from model outputs
#     attentions = outputs[3]  # This is a tuple of attention scores from different layers

#     # Return tokenized input and attention scores from the last layer
#     return inputs, attentions[-1]  # Taking the last layer's attention weights

# def explain_token_attention(tokenizer, inputs, attention_scores):
#     input_ids = inputs['input_ids'][0]  # First sequence in batch
#     tokens = tokenizer.convert_ids_to_tokens(input_ids)

#     # Average the attention scores across heads for simplicity
#     attention_mean = attention_scores.mean(dim=1)  # Average over all heads

#     # Get attention weights for each token
#     explanations = {}
#     for i, token in enumerate(tokens):
#         token_attention = attention_mean[:, i].mean().item()  # Average over source tokens
#         explanations[token] = token_attention

#     return explanations

# # Example usage
# text = "you are such a bitch"
# inputs, attention_scores = get_attention_scores(loaded_model, tokenizer, text)
# explanations = explain_token_attention(tokenizer, inputs, attention_scores)

# # Print token-wise explanations
# for token, score in explanations.items():
#     print(f"Token: {token}, Attention: {score}")